
# TuIT_DeepRL - DRL Project
![OST](./resources/ost_logo.png)
## MiniF1RL: My pygame 2d racing environment for DRL
Author: Lars Herrmann    
Date: 12.04.2024  
Repository: [Github - miniF1RL](https://github.com/lherrman/miniF1RL)   
Python Version: 3.11

## Introduction
After a lot of experimentaition and consideration of what environment to use. (Playing with e.g. TrackManiaRL, Gymnasium Integrated Environemnts and Unity Environments), i decided to implement my own 2d racing environment. The main reason for this is that i wanted a environment where no pre-trained models are available what motivates me to train my own models. As i started watching formula one this season, i wanted to go with a racing environment. 

## Implementing the environment

### The CarModel
As a starting point, i was able to use a Python class of a simple 2d topdown car model that i implemented for another project about a year ago. The class already had methods to draw a simple 2d box car and it's wheels onto a pygame screen. As it wasn't intended as a racing game, and rather a geometrical model to controll an rc car, i had to implement an updated method for updating the cars position and velocity to make it more fun to drive. The original CarModel class can be found in my github repository for the [SlamCar Project](https://github.com/lherrman/slamcar-controller) in the file `base_model.py`. 

To make this into a racing environment, there were still some parts missing i startet to implement, starting with a track to drive on.

As of now, not a lot of the original code is left.

### The Track
In order to have a track to drive on, i decided to draw a black and white image of a track in Photoshop. When initializing the car model, i wrote a python method that uses the 'cv2.findContours' method to find the track boundaries in the image. The track is then represented as two lists of points, one for the inner and one for the outer boundary.

![Track Image](./resources/MakeTrack.png)


Python Code:
```python
    def _get_track_boundaries_from_image(self, image_path) -> dict:
        '''
        Using opencv to read the image and find the contours of the track boundaries.
        The boundaries are represented as list of points. ([x1, y1], [x2, y2], ...)
        '''
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        contours, _ = cv2.findContours(image.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        assert len(contours) == 2, "There should be exactly 2 contours in the image"

        # Transform and downsample the contours
        downsample_rate = 10
        inner_boundary = contours[0][:, 0, :][::downsample_rate]
        outer_boundary = contours[1][:, 0, :][::downsample_rate]

        # # Append last point to close the loop
        inner_boundary = np.append(inner_boundary, [inner_boundary[0]], axis=0)
        outer_boundary = np.append(outer_boundary, [outer_boundary[0]], axis=0)

        # Scale the contours to make suitable for the simulation
        inner_boundary, outer_boundary = inner_boundary / 60, outer_boundary / 60

        return {
            'inner': inner_boundary,
            'outer': outer_boundary
        }   
```

#### Track progress
To approximate the progress the car has made on the track, i implemented a method `_calculate_track_progress` that calculates the distance to the next point on the inner boundary. The first point on the inner boundary is set as the starting point. The progress is then calculated as the index of the nearest point divided by the total number of points on the inner boundary. This could be improved by taking the length of the segments passed into account.

### The observation space

I wanted to keep the observation space simple, so i decided to use 'lidar sensors', that give the car only a few distance measurements in front of it. I decided to use 3 sensors, one in the middle and one on each side of the car in a 45 degree angle. The sensors are implemented as a method of the CarModel class that returns the distances to the track boundaries. 

The implementation of the `_segment_intersection` and `_raycast` methods was prompted to ChatGPT, which returned an working implementation after 2-3 iterations. The method uses an all python implementation for which the performance isn't optimal, but i guess it's good enough for my purposes. As the raycast method has to itterate over each segment of the boundaries, a more performant implementation would be very beneficial.


For easier validation, and because it looks cool, i decided to draw the lidar sensors as lines on the pygame screen.


![Lidar Sensors](./resources/lidars.png)

In addition to the 3 lidar sensors, i also added the cars steering angle to the observation space.


#### Update (16.05.2024):  
I implemented a more performant version of the raycast method using numpy and numba again with the help of a chatbot.

This sped up the maximum number of ticks per second from  about 230-300 to about 1000.

Measurements 

New Method:  
```
FPS: 59.53  | Ticks: 1001.00 | Tick Time: 0:00:00.999000 ms
```
Old Method:  
```
FPS: 52.12  | Ticks: 247.83 | Tick Time: 0:00:04.035000 ms
```


### The action space

The action space is also very simple. To make the problem even easier for the RL algorithm, i decided to always have the car accelerate, as slowing down isn't necessary in this environment. The car can steer left or right, or go straight. To make it a little more interesting, i added a third action, boost, that allows the car to go faster on the straights.

### The reward function

I decided the use the following 4 types of rewards:
- Progress reward: The car gets a reward for making progress on the track
- Speed reward: The car gets a reward for driving fast
- Finish reward: The car gets a reward for finishing the track
- Collision reward: The car gets a negative reward for colliding with the track boundaries

As the rewards are calculated from differently scaled values, the weigths of progress and speed can't be compared directly. After some testing, i decided to use the following weights for the rewards:
- Progress: 1000 (* fracton of the progress made in the last step)
- Speed: 1 (* speed in m/s)
- Finish: 100 (* 1 if the car has finished the track)
- Collision: -100 (* 1 if the car has collided with the track boundaries)

### Adding Graphics
To enhance the graphics of the environment, i generated a background image based on the drawn track by locally running a stable diffusion model and prompting it with the track image and a text description of the track i want.

I used the JuggernautXL model trough the [Fooocus UI](https://github.com/lllyasviel/Fooocus).  
I got the best results by setting the image prompt mode to CPDS, in order to keep the structure of the track. By decreasing the weight of the image prompt, the track structure is kept, but the model is free to generate a more interesting background.

![Image Gen](./resources/fooocus_image_gen.png)

Also for the car i generated a sprite image that is used to draw the car on the screen.   
Note: I also got some quite funny results:  
![Car Gen](./resources/car_fail.png)

The environment allows to switch between the simple wireframe graphics and the enhanced graphics by pressing the 'g' key.
I also added the possibility to zoom in and out using the mouse wheel. When the whole track is visible the camera will center on the track instead of the car.

## Demo animation
![Demo Animation](./resources/Trained_Env_Demo.gif)

## Helper functions

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import threading
import torch
import pygame as pg 
from minif1env import MiniF1RLEnv
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_util import make_vec_env
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3.common.callbacks import CheckpointCallback
import os
import glob

def get_writer(texts: dict):
    writer = SummaryWriter()
    for key, value in texts.items():
        writer.add_text(str(key), str(value))
    return writer

# Use a separate thread to render the environment
# Avoids crashing the pg window during training
def render_loop(env: MiniF1RLEnv, 
           close_event, 
           human_control=False, 
           fps=60):
    pg.init()
    while True:
        delay_ms = 1000 // fps
        env.render()
        pg.time.wait(delay_ms) 

        done = env.handle_pg_events(human_control=human_control)

        if done:
           close_event.set()

        if close_event.is_set():
            pg.quit()
            return
        
def latest_checkpoint_file_path(model_description: str):
  checkpoint_dir = "checkpoints"
  checkpoint_files =  glob.glob(os.path.join(checkpoint_dir, f"*{model_description}*/*.zip"))
  if not checkpoint_files:
    return None
  latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
  return latest_checkpoint


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Manual Play with the environment


In [4]:
env = MiniF1RLEnv()

done = False
last_frame = datetime.now()
while not done:

    # Limit framerate for manual controlls
    if (datetime.now() - last_frame).total_seconds() < 1/100:
        continue
    last_frame = datetime.now()

    exit = env.handle_pg_events(human_control=True)

    observation, step_reward, terminate, truncate, info = env.step(None)
    env.render()

    if terminate:
        env.reset()

    if exit:
        break

env.close()


# Training

In [8]:


# Create the environment
env = MiniF1RLEnv()

# Create and start the rendering thread
close_event = threading.Event()
render_thread = threading.Thread(target=render_loop, args=(env,close_event,False,30))
render_thread.start()

# Set the training parameters
total_timesteps = 1_000_000
agent_description = "PPO"
policy_decription = "MlpPolicy"
model_description = f"{agent_description}_{policy_decription}_128_64_A5"
print(f"Training model: {model_description}")

# Define the policy network and check if GPU is available
policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[128, 64])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


load_checkpoint = True
# Load the latest checkpoint if available, otherwise create a new model
latest_checkpoint = latest_checkpoint_file_path(model_description)
if load_checkpoint and latest_checkpoint:
  print(f"Loading checkpoint: {latest_checkpoint}")
  model = PPO.load(latest_checkpoint, env, verbose=1)
else:
  model = PPO(policy_decription, env, verbose=1, tensorboard_log=f"./logs", policy_kwargs=policy_kwargs, device=device)
run_name = datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + model_description


# # Tensorboard writer
# writer = get_writer({"run_name": run_name,
#                     "model_description": model_description,
#                     "total_timesteps": total_timesteps,
#                     "env_name": "MiniF1RLEnv",
#                     "reward_weights": env.get_reward_weights(),
#                     "device": device,
#                     "policy_kwargs": policy_kwargs})

# Save a checkpoint every 10000 steps
tb_logs = f"./logs/{run_name}"
checkpoint_callback = CheckpointCallback(
  save_freq=10_000,
  save_path=f"./checkpoints/{run_name}",
  name_prefix=f"{run_name}_checkpoint",
  save_replay_buffer=False,
  save_vecnormalize=True,
)

# Train the model
model.learn(total_timesteps=total_timesteps, 
            tb_log_name=run_name, 
            callback=checkpoint_callback)
model.save(f"models/{run_name}")

# Close the rendering thread and environment
close_event.set()
render_thread.join()
env.close()

# Close the tensorboard writer
#writer.close()

Training model: PPO_MlpPolicy_128_64_A5
Using device: cuda
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs\20240603_201536_PPO_MlpPolicy_128_64_A5_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.1     |
|    ep_rew_mean     | -90.3    |
| time/              |          |
|    fps             | 198      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 89.9        |
|    ep_rew_mean          | -99.2       |
| time/                   |             |
|    fps                  | 177         |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            

KeyboardInterrupt: 

# Inference


In [6]:

#model_description = "PPO_MlpPolicy_64_64_R7"
env = MiniF1RLEnv()

# Load the latest checkpoint that matches the model description
latest_checkpoint = latest_checkpoint_file_path(model_description)
if not latest_checkpoint:
    raise ValueError(f"No checkpoint found for model description: {model_description}")
print(f"Loading checkpoint: {latest_checkpoint}")

model = PPO.load(latest_checkpoint)

# Create and start the rendering thread
close_event = threading.Event()
render_thread = threading.Thread(target=render_loop, args=(env,close_event,False,30))
render_thread.start()

# Run the model
exit = False
while not exit:
    obs, info = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, truncate, info = env.step(action)
    
        if close_event.is_set():
            exit = True
            break

# Close the rendering thread and environment
close_event.set()
render_thread.join()
env.close()



Loading checkpoint: checkpoints\20240603_121512_PPO_MlpPolicy_64_64_A3\20240603_121512_PPO_MlpPolicy_64_64_A3_checkpoint_1000000_steps.zip


# Training Log
These are rougly the different stages at which i run the training.

## Tests 01
**Best Model:**  "a2c_minif1rl"   
**Timesteps**: 10'000   
**Total Training Time:**  ~1h  
**Algorithm:** A2C  
**Conditions:** After every crash, the car is reset to the starting position.
**Result:** The car learned to always directly drive into the right border.     
Explaination. After checking the rewards that were given to the model, i realized that the reward function was not working as intended. The weight for the progress reward was way to low so the model did not learn to follow the track. Probebly also the punishment for hitting the track boundaries was to low.

## Tests 02
**Best Model:**  "PPO_MlpPolicy_64_64_R4"   
**Timesteps**: 10'000  
**Total Training Time:**  ~1h   
**Algorithm:** PPO  
**Conditions:** After every crash, the car is reset to the starting position. I added some slight randomness to the starting position and heading of the car.   
**Result:** Still in most of the runs the car learned to drive into the right border. He never came to the point to get to the first left turn.

## Tests 03
**Best Model:**  "PPO_MlpPolicy_64_64_R7"   
**Timesteps**: 10'000  
**Total Training Time:**  ~1h  
**Algorithm:** PPO  
**Conditions:** (New reset function implemented) After every crash, the car is reset to a random position in the center of the track heading into track direction.    
**Result:** The car learned to drive on the track. On the best run, after about 10 minutes it learned to do the right curves pretty consistently. The left curves were still a problem until about 30 minutes into the training. I experience quite a large difference in learning speed between the different runs. Some runs never learned to drive on the track properly as the started to consistenlty drive into the right border.

# Training Runs V2
As is updated the environment wiht new configurations, i descided to restart my training process. I added better logging to TensorBoard to better track the progress.

This time, every run in a full run of 1'000'000 timesteps logged to TensorBoard.

## PPO_MlpPolicy_64_64_A2
Timesteps: 1'000'000    
Total Training Time:  ~110min   
Algorithm: PPO   
Policy: MlpPolicy with 64 hidden units and 64 hidden units   

Reward Multipliers:
- Progress: 1000
- Speed: 1
- Finish: 100
- Collision: -200

Reward discount factor: 0.999  


## PPO_MlpPolicy_64_64_A3
Timesteps: 1'000'000    
Total Training Time:  ~110min   
Algorithm: PPO   
Policy: MlpPolicy with 64 hidden units and 64 hidden units   

Reward Multipliers:
- Progress: 1000
- Speed: 1
- Finish: 100
- Collision: -200

Reward discount factor: 0.99  
